### Entregable 1

In [1]:
import requests
import json
response_API = requests.get("https://chilealerta.com/api/query/?user=demo&select=ultimos_sismos&limit=100&country=Chile")
print(response_API.status_code)

200


In [2]:
#JSON a diccionario
ultimos_sismos = json.loads(response_API.text)
ultimos_sismos = ultimos_sismos['ultimos_sismos_Chile']
ultimos_sismos

[{'state': 1,
  'utc_time': '2023/03/23 21:24:11',
  'local_time': '2023-03-23 18:24:11',
  'chilean_time': '2023/03/23 18:24:11',
  'reference': '159 km al E de Chillán - Chile',
  'magnitude': 4.5,
  'scale': 'Mb',
  'latitude': -36.69,
  'longitude': -70.33,
  'depth': 157,
  'id': '26060595',
  'url': 'http://sismologia.net/?p=detalles&id=26060595',
  'source': 'INSIMU'},
 {'state': 1,
  'utc_time': '2023/03/23 05:43:53',
  'local_time': '2023-03-23 02:43:53',
  'chilean_time': '2023/03/23 02:43:53',
  'reference': '61 km al E de Rancagua - Chile',
  'magnitude': 4.2,
  'scale': 'Mb',
  'latitude': -34.11,
  'longitude': -70.09,
  'depth': 129,
  'id': '82068640',
  'url': 'http://sismologia.net/?p=detalles&id=82068640',
  'source': 'INSIMU'},
 {'state': 1,
  'utc_time': '2023/03/22 15:38:37',
  'local_time': '2023-03-22 12:38:37',
  'chilean_time': '2023/03/22 12:38:37',
  'reference': '171 km al NE de Antofagasta - Chile',
  'magnitude': 4.5,
  'scale': 'Mb',
  'latitude': -22.72

### Entregable 2

In [3]:
#!pip install python-dotenv

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os 

url= os.getenv('URL')
data_base= os.getenv('DATA_BASE')
user= os.getenv('USER')
pwd= os.getenv('PWD')
port = os.getenv('PORT')

In [6]:
# Creando la conexión a Redsshift
import psycopg2
try:
    conn = psycopg2.connect(
        host=url,
        dbname=data_base,
        user=user,
        password=pwd,
        port=port
    )
    print("¡Conexión existosa!")
    
except Exception as e:
    print("¡Conexión invalida!", e)

¡Conexión existosa!


In [7]:
#Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS sfparevalo_coderhouse.earthquake (
            id INTEGER primary key,
            state_earthquake INTEGER,
            utc_time TIMESTAMP,
            reference VARCHAR(255),
            magnitude DECIMAL(5, 2),
            scale_earthquake VARCHAR(5),
            latitude DECIMAL(8, 4),
            longitude DECIMAL(8, 4),
            depth_earthquake DECIMAL(8, 2)
        );
    """)
    conn.commit()

In [8]:
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
    cur.execute("Truncate table earthquake")
    count = cur.rowcount

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(ultimos_sismos)

In [11]:
#Eliminar elementos duplicados
df.drop_duplicates(subset=['id'], inplace=True)

In [12]:
#Verificar que la fecha se muestre en formato fecha 
df['utc_time']= pd.to_datetime(df['utc_time'], format='%Y-%m-%d %H:%M:%S')

In [13]:
df.isnull().sum()

state           0
utc_time        0
local_time      0
chilean_time    0
reference       0
magnitude       0
scale           0
latitude        0
longitude       0
depth           0
id              0
url             0
source          0
dtype: int64

In [14]:
#Considerar datos que se van a guardar en la base de datos.
df = df[['id', 'state', 'utc_time', 'reference', 'magnitude', 'scale', 'latitude', 'longitude', 'depth']]
df.head()

,id,state,utc_time,reference,magnitude,scale,latitude,longitude,depth
0,26060595,1,2023-03-23 21:24:11,159 km al E de Chillán - Chile,4.5,Mb,-36.6900,-70.3300,157.00
1,82068640,1,2023-03-23 05:43:53,61 km al E de Rancagua - Chile,4.2,Mb,-34.1100,-70.0900,129.00
2,43003785,1,2023-03-22 15:38:37,171 km al NE de Antofagasta - Chile,4.5,Mb,-22.7200,-69.0600,110.00
3,34055540,1,2023-03-22 13:36:08,113 km al NO de La Serena - Chile,4.4,Mb,-29.6900,-72.3900,10.00
4,88705756,2,2023-03-21 17:38:13,68 km al SO de Santiago de Chile - Chile,5.6,Mb,-33.6103,-71.3517,65.05


In [15]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO earthquake (id, state_earthquake, utc_time, reference, magnitude, scale_earthquake, latitude, longitude, depth_earthquake)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [16]:
#Cerrar conexión
cur.close()
conn.close()